# Monitor Function Memory Consumption in Python

The idea is to have a function decorator that tracks the memory consumption of a function and stops the function in case a limit is reached.

Based on stackoverflow answer: https://stackoverflow.com/a/10117657

In [1]:
import functools
import os
import psutil
import threading
import time

## Memory Resource Monitor

In [2]:
class MemoryMonitor(threading.Thread):
    def __init__(self, target_function, *args, **kwargs):
        super(MemoryMonitor, self).__init__()
        self.target_function = target_function
        self.args = args
        self.kwargs = kwargs
        self.results = None
        self.daemon = True
        self.event = threading.Event()
        self.event.set()
        self.__has_shutdown = False

    def run(self):
        """ Overloads the threading.Thread.run """

        # Call the User's Startup functions
        self.startup()

        # Loop until the thread is stopped
        while self.is_running:
            self.mainloop()

        # Clean up
        self.cleanup()
        self.__has_shutdown = True

    def stop(self):
        self.event.clear()
    
    @property
    def is_running(self):
        return self.event.is_set()

    @property
    def is_shutdown(self):
        return self.__has_shutdown

    def startup(self):
        print(f'Monitoring {self.target_function.__name__} function')

    def cleanup(self):
        print(f'{self.target_function.__name__} call complete')

    def mainloop(self):
        # Start the library Call
        self.results = self.target_function(*self.args, **self.kwargs)

        # Kill the thread when complete
        self.stop()

## Testing the monitor function

### Define a testing function that uses alot of memory

In [3]:
def memory_hog(set_size, num_dict_entries, delay_per_entry=0):
    """ This function creates a dictionary with 'num_dict_entries' of a set 
    of size 'set_size'
    """
    large_dictionary = {}

    for i in range(num_dict_entries):
        time.sleep(delay_per_entry)
        large_dictionary[f'entry_{i}'] = set(range(set_size))

    return len(large_dictionary)

# out = memory_hog(10000, 15000) 

### run memory monitor

In [4]:
process = psutil.Process(os.getpid())
start_mem = process.memory_full_info().uss

mythread = MemoryMonitor(memory_hog, 10000, 5000)
mythread.start()

delta_mem = 0
max_memory = 0

while True:
    delta_mem = process.memory_full_info().uss - start_mem
    if delta_mem > max_memory:
        max_memory = delta_mem

    # Check to see if the library call is complete
    if mythread.is_shutdown:
        break

print(f'MAX Memory Usage in MB: {max_memory / (1024.0 ** 2):.3f}')

Monitoring memory_hog function
memory_hog call complete
MAX Memory Usage in MB: 3711.160


In [6]:
# reset memory in testing
# del mythread

## Using it as function decorator

In [5]:
def mem_monitor(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        process = psutil.Process(os.getpid())
        start_mem = process.memory_full_info().uss

        func_thread = MemoryMonitor(func, *args, **kwargs)
        func_thread.start()

        delta_mem, max_memory = 0, 0
        while True:
            delta_mem = process.memory_full_info().uss - start_mem
            if delta_mem > max_memory:
                max_memory = delta_mem

            # Check to see if the library call is complete
            if func_thread.is_shutdown:
                break

        print(f'Max Memory Usage in MB: {max_memory / (1024.0 ** 2):.3f}')
        return func_thread.results
    return wrapper

In [6]:
@mem_monitor
def memory_hog_decorated(set_size, num_dict_entries, delay_per_entry=0):
    """ This function creates a dictionary with 'num_dict_entries' of a set 
    of size 'set_size'
    """
    large_dictionary = {}

    for i in range(num_dict_entries):
        time.sleep(delay_per_entry)
        large_dictionary[f'entry_{i}'] = set(range(set_size))

    return len(large_dictionary)

memory_hog_decorated(10000, 5000) 

Monitoring memory_hog_decorated function
memory_hog_decorated call complete
Max Memory Usage in MB: 3779.367


5000